In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
from rdkit import Chem

In [2]:
output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex"

In [3]:
data_path = "/home/icb/dominik.klein/git_repos/ot_pert_new/competing_methods/combosciplex/CPA/combo_sciplex_prep_hvg_filtered.h5ad"
adata = sc.read(data_path)

/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [4]:
adata

AnnData object with n_obs × n_vars = 63378 × 5000
    obs: 'ncounts', 'well', 'plate', 'cell_line', 'replicate', 'time', 'dose_value', 'pathway1', 'pathway2', 'perturbation', 'target', 'pathway', 'dose_unit', 'celltype', 'disease', 'tissue_type', 'organism', 'perturbation_type', 'ngenes', 'percent_mito', 'percent_ribo', 'nperts', 'chembl-ID', 'condition', 'condition_ID', 'control', 'cell_type', 'smiles_rdkit', 'source', 'sample', 'Size_Factor', 'n.umi', 'RT_well', 'Drug1', 'Drug2', 'Well', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'split', 'condition_old', 'pert_type', 'batch', 'split_1ct_MEC', 'split_2ct_MEC', 'split_3ct_MEC', 'batch_cov', 'batch_cov_cond', 'log_dose', 'cov_drug_dose'
    var: 'ensembl_id-0', 'ncounts-0', 'ncells-0', 'symbol-0', 'symbol-1', 'id-1', 'n_cells-1', 'mt-1', 'n_cells_by_counts-1', 'mean_counts-1', 'pct_dropout_by_counts-1', 'total_counts-1', 'highly_variable-1', 'means-1', 'dispersions-1', 'dispersions_nor

In [5]:
adata.obs["condition"].value_counts()

condition
Dacinostat (LAQ824)+PCI-34051                       3298
SRT3025+Cediranib (AZD2171)                         3016
Givinostat (ITF2357)+Cediranib (AZD2171)            2783
SRT2104 (GSK2245840)                                2756
Givinostat (ITF2357)+Curcumin                       2736
Givinostat (ITF2357)+Sorafenib Tosylate             2734
Givinostat (ITF2357)+Carmofur                       2692
Givinostat (ITF2357)+Crizotinib (PF-02341066)       2662
Givinostat (ITF2357)+Dasatinib                      2421
Givinostat (ITF2357)+SRT2104 (GSK2245840)           2353
Dasatinib                                           2343
Givinostat (ITF2357)+SRT1720                        2260
Panobinostat (LBH589)+Curcumin                      2244
Cediranib (AZD2171)+PCI-34051                       2161
Panobinostat (LBH589)+Sorafenib Tosylate            2013
Panobinostat (LBH589)+SRT2104 (GSK2245840)          1971
Panobinostat (LBH589)+Dasatinib                     1955
Dacinostat (LAQ824)+D

In [6]:
adata.obs["Drug2"].value_counts()

Drug2
Dasatinib       7950
PCI-34051       7273
SRT2104         7080
Cediranib       5799
Curcumin        4980
Sorafenib       4747
Crizotinib      4303
SRT1720         4086
Carmofur        2692
Alvespimycin    2274
Danusertib      1939
SRT3025         1889
Dacinostat      1869
Givinostat      1682
Panobinostat    1578
control         1451
Tanespimycin    1310
Pirarubicin      476
Name: count, dtype: int64

In [7]:
adata.obs["Drug1"].value_counts()

Drug1
Givinostat      21951
Panobinostat    16349
control         12437
Dacinostat       6468
SRT3025          3016
Cediranib        2161
SRT2104           520
Alvespimycin      476
Name: count, dtype: int64

In [8]:
adata.X = adata.layers["counts"]

In [9]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)

In [10]:
sc.pp.highly_variable_genes(adata, inplace=True, n_top_genes=2000)

/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/scanpy/preprocessing/_highly_variable_genes.py:226: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby("mean_bin")["dispersions"]


In [11]:
adata = adata[:, adata.var["highly_variable"] == True]

In [12]:
adata.obs["condition"] = adata.obs.apply(
    lambda x: "control" if x["condition"] == "control" else x["Drug1"]+"_"+x["Drug2"], axis=1
)

/tmp/ipykernel_579898/3265058569.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs["condition"] = adata.obs.apply(


In [13]:
adata.obs["condition"]

A01_A02_RT_BC_10_Lig_BC_18-1    control_Panobinostat
A01_A02_RT_BC_10_Lig_BC_25-1    control_Panobinostat
A01_A02_RT_BC_10_Lig_BC_48-1    control_Panobinostat
A01_A02_RT_BC_10_Lig_BC_51-1    control_Panobinostat
A01_A02_RT_BC_10_Lig_BC_70-1    control_Panobinostat
                                        ...         
H12_A02_RT_BC_96_Lig_BC_50-1       control_Dasatinib
H12_A02_RT_BC_96_Lig_BC_95-1       control_Dasatinib
H12_A02_RT_BC_9_Lig_BC_41-1     Dacinostat_Dasatinib
H12_A02_RT_BC_9_Lig_BC_69-1     Dacinostat_Dasatinib
H12_A02_RT_BC_9_Lig_BC_83-1     Dacinostat_Dasatinib
Name: condition, Length: 63378, dtype: object

In [14]:
def rank_genes_groups_by_cov(
    adata,
    groupby,
    control_group,
    covariate,
    n_genes=50,
    rankby_abs=True,
    key_added="rank_genes_groups_cov",
    return_dict=False,
):
    gene_dict = {}
    cov_categories = adata.obs[covariate].unique()
    for cov_cat in cov_categories:
        # name of the control group in the groupby obs column
        control_group_cov = control_group  # "_".join([cov_cat, control_group])
        # subset adata to cells belonging to a covariate category
        adata_cov = adata[adata.obs[covariate] == cov_cat]
        # compute DEGs
        sc.tl.rank_genes_groups(
            adata_cov,
            groupby=groupby,
            reference=control_group_cov,
            rankby_abs=rankby_abs,
            n_genes=n_genes,
            use_raw=False,
        )
        # add entries to dictionary of gene sets
        de_genes = pd.DataFrame(adata_cov.uns["rank_genes_groups"]["names"])
        for group in de_genes:
            gene_dict[group] = de_genes[group].tolist()
    adata.uns[key_added] = gene_dict
    if return_dict:
        return gene_dict


def get_DE_genes(adata):
    adata.obs.loc[:, "control"] = adata.obs.condition.apply(lambda x: 1 if x == "control" else 0)
    adata.obs = adata.obs.astype("category")
    rank_genes_groups_by_cov(
        adata,
        groupby="condition",
        covariate="cell_type",
        control_group="control",
        n_genes=50,
        key_added="rank_genes_groups_cov_all",
    )
    return adata

In [15]:
adata = get_DE_genes(adata)

/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/scanpy/tools/_rank_genes_groups.py:396: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, 'names'] = self.var_names[global_indices]
/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/scanpy/tools/_rank_genes_groups.py:398: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider j

In [16]:
assert "rank_genes_groups_cov_all" in adata.uns.keys()

In [17]:
# taken from pubchem
drug_to_smiles = {
    "Panobinostat": "CC1=C(C2=CC=CC=C2N1)CCNCC3=CC=C(C=C3)/C=C/C(=O)NO",
    "PCI-34051": "COC1=CC=C(C=C1)CN2C=CC3=C2C=C(C=C3)C(=O)NO",
    "SRT1720": "C1CN(CCN1)CC2=CSC3=NC(=CN23)C4=CC=CC=C4NC(=O)C5=NC6=CC=CC=C6N=C5",
    "SRT3025": "COCCCC1=C(N=C(S1)C2=CC=CC=C2)C(=O)NC3=CC=CC=C3C4=NC5=C(S4)N=CC(=C5)CN6CCCC6",
    "Dacinostat": "C1=CC=C2C(=C1)C(=CN2)CCN(CCO)CC3=CC=C(C=C3)/C=C/C(=O)NO",
    "Sorafenib": "CNC(=O)C1=NC=CC(=C1)OC2=CC=C(C=C2)NC(=O)NC3=CC(=C(C=C3)Cl)C(F)(F)F",
    "Cediranib": "CC1=CC2=C(N1)C=CC(=C2F)OC3=NC=NC4=CC(=C(C=C43)OC)OCCCN5CCCC5",
    "Givinostat": "CCN(CC)CC1=CC2=C(C=C1)C=C(C=C2)COC(=O)NC3=CC=C(C=C3)C(=O)NO",
    "Danusertib": "CN1CCN(CC1)C2=CC=C(C=C2)C(=O)NC3=NNC4=C3CN(C4)C(=O)[C@@H](C5=CC=CC=C5)OC",
    "Dasatinib": "CC1=C(C(=CC=C1)Cl)NC(=O)C2=CN=C(S2)NC3=CC(=NC(=N3)C)N4CCN(CC4)CCO",
    "Tanespimycin": r"C[C@H]1C[C@@H]([C@@H]([C@H](/C=C(/[C@@H]([C@H](/C=C\C=C(\C(=O)NC2=CC(=O)C(=C(C1)C2=O)NCC=C)/C)OC)OC(=O)N)\C)C)O)OC",
    "Carmofur": "CCCCCCNC(=O)N1C=C(C(=O)NC1=O)F",
    "SRT2104": "CC1=C(SC(=N1)C2=CN=CC=C2)C(=O)NC3=CC=CC=C3C4=CN5C(=CSC5=N4)CN6CCOCC6",
    "Crizotinib": "C[C@H](C1=C(C=CC(=C1Cl)F)Cl)OC2=C(N=CC(=C2)C3=CN(N=C3)C4CCNCC4)N",
    "Pirarubicin": "C[C@H]1[C@H]([C@H](C[C@@H](O1)O[C@H]2C[C@@](CC3=C2C(=C4C(=C3O)C(=O)C5=C(C4=O)C(=CC=C5)OC)O)(C(=O)CO)O)N)O[C@@H]6CCCCO6",
    "Alvespimycin": r"C[C@H]1C[C@@H]([C@@H]([C@H](/C=C(/[C@@H]([C@H](/C=C\C=C(\C(=O)NC2=CC(=O)C(=C(C1)C2=O)NCCN(C)C)/C)OC)OC(=O)N)\C)C)O)OC",
    "Curcumin": "COC1=C(C=CC(=C1)/C=C/C(=O)CC(=O)/C=C/C2=CC(=C(C=C2)O)OC)O",
    "control": None,
}

In [18]:
adata.obs["smiles_drug_1"] = adata.obs["Drug1"].map(drug_to_smiles)
adata.obs["smiles_drug_2"] = adata.obs["Drug2"].map(drug_to_smiles)

In [19]:
from rdkit.Chem import AllChem


def get_fp(smiles, radius=4, nBits=1024):
    m = Chem.MolFromSmiles(smiles, sanitize=False)
    if m is None:
        return "invalid"
    else:
        try:
            Chem.SanitizeMol(m)
        except:
            return "invalid"
    return AllChem.GetHashedMorganFingerprint(m, radius=radius, nBits=nBits)

In [20]:
drug_to_fp = {}
for drug, sm in drug_to_smiles.items():
    if not isinstance(sm, str):
        continue
    drug_to_fp[drug] = np.array(list(get_fp(sm)))

In [21]:
len(drug_to_fp), len(drug_to_smiles)

(17, 18)

In [22]:
features_df = pd.DataFrame.from_dict(drug_to_fp).T

In [23]:
features_df

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
Panobinostat,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
PCI-34051,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
SRT1720,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
SRT3025,0,0,1,0,2,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
Dacinostat,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Sorafenib,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,1,0,0,0,0
Cediranib,0,0,0,0,2,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,1
Givinostat,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Danusertib,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Dasatinib,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
features_cells_drug_1 = np.zeros((adata.shape[0], features_df.shape[1]))
features_cells_drug_2 = np.zeros((adata.shape[0], features_df.shape[1]))
for mol, ecfp in features_df.iterrows():
    print(mol)
    features_cells_drug_1[adata.obs["Drug1"].isin([mol])] = ecfp.values
    features_cells_drug_2[adata.obs["Drug2"].isin([mol])] = ecfp.values

Panobinostat
PCI-34051
SRT1720
SRT3025
Dacinostat
Sorafenib
Cediranib
Givinostat
Danusertib
Dasatinib
Tanespimycin
Carmofur
SRT2104
Crizotinib
Pirarubicin
Alvespimycin
Curcumin


In [25]:
adata.obsm["ecfp_drug_1"] = features_cells_drug_1
adata.obsm["ecfp_drug_2"] = features_cells_drug_2

In [26]:
adata.obs['split_1ct_MEC'].value_counts()

split_1ct_MEC
train    49683
ood       8209
valid     5486
Name: count, dtype: int64

In [27]:
adata[adata.obs['split_1ct_MEC']=="valid"].obs["condition"].value_counts()

condition
Dacinostat_PCI-34051        323
SRT3025_Cediranib           293
Givinostat_Curcumin         289
Givinostat_Crizotinib       279
Givinostat_Carmofur         267
control_SRT2104             265
Givinostat_Dasatinib        263
Givinostat_Sorafenib        255
Panobinostat_Curcumin       228
Givinostat_SRT2104          226
Givinostat_SRT1720          223
Cediranib_PCI-34051         211
control_Dasatinib           210
Panobinostat_Sorafenib      203
Panobinostat_SRT1720        195
Panobinostat_Dasatinib      193
Panobinostat_SRT3025        192
control_Dacinostat          190
Panobinostat_PCI-34051      185
Panobinostat_Crizotinib     169
control_Panobinostat        165
control_Givinostat          155
control                     142
Givinostat_Tanespimycin     135
Dacinostat_Dasatinib        107
control_Alvespimycin         68
Alvespimycin_Pirarubicin     55
Name: count, dtype: int64

In [28]:
import anndata

adata_train_final = adata[adata.obs['split_1ct_MEC']=="train"]
adata_test_final = adata[adata.obs['split_1ct_MEC']=="valid"]
adata_ood = adata[adata.obs['split_1ct_MEC']=="ood"]
adata_ood_final = adata_ood.concatenate(adata_test_final[adata_test_final.obs["condition"]=="control"])

/tmp/ipykernel_579898/3308937776.py:6: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  adata_ood_final = adata_ood.concatenate(adata_test_final[adata_test_final.obs["condition"]=="control"])


In [29]:
adata_train_final.n_obs, adata_test_final.n_obs, adata_ood_final.n_obs 

(49683, 5486, 8351)

In [30]:
adata_train_final.varm["X_train_mean"] = adata_train_final.X.mean(axis=0).T

/tmp/ipykernel_579898/2775379558.py:1: ImplicitModificationWarning: Setting element `.varm['X_train_mean']` of view, initializing view as actual.
  adata_train_final.varm["X_train_mean"] = adata_train_final.X.mean(axis=0).T


In [31]:
from scipy.sparse import csr_matrix

adata_train_final.layers["X_log1p"] = adata_train_final.X.copy()
train_mean = adata_train_final.varm["X_train_mean"].T
adata_train_final.layers["centered_X"] = csr_matrix(adata_train_final.X.A - train_mean)
adata_train_final.X = adata_train_final.layers["centered_X"]

In [32]:
sc.pp.pca(adata_train_final, zero_center=False, n_comps=30)
adata_train_final.X = adata_train_final.layers["X_log1p"]

In [33]:
adata_train_final.obsm["X_pca"].shape

(49683, 30)

In [34]:
adata_ood_final.varm["X_train_mean"] = adata_train_final.varm["X_train_mean"]
adata_test_final.varm["X_train_mean"] = adata_train_final.varm["X_train_mean"]

/tmp/ipykernel_579898/3355210628.py:2: ImplicitModificationWarning: Setting element `.varm['X_train_mean']` of view, initializing view as actual.
  adata_test_final.varm["X_train_mean"] = adata_train_final.varm["X_train_mean"]


In [35]:
adata_test_final.layers["centered_X"] = csr_matrix(adata_test_final.X.A - train_mean)
adata_ood_final.layers["centered_X"] = csr_matrix(adata_ood_final.X.A - train_mean)

In [36]:
adata_test_final.obsm["X_pca"] = np.matmul(adata_test_final.layers["centered_X"].A, adata_train_final.varm["PCs"])
adata_ood_final.obsm["X_pca"] = np.matmul(adata_ood_final.layers["centered_X"].A, adata_train_final.varm["PCs"])

In [37]:
adata_train_final.uns.keys()

dict_keys(['cell_type_colors', 'hvg', 'neighbors', 'pathway1_colors', 'pca', 'rank_genes_groups_cov', 'rank_genes_groups_cov_detailed', 'source_colors', 'umap', 'log1p', 'rank_genes_groups_cov_all'])

In [38]:
adata_train_final.varm["X_train_mean"] = np.asarray(adata_train_final.varm["X_train_mean"])
adata_test_final.varm["X_train_mean"] = np.asarray(adata_test_final.varm["X_train_mean"])
adata_ood_final.varm["X_train_mean"] = np.asarray(adata_ood_final.varm["X_train_mean"])

In [39]:
adata_ood_final.uns = adata_train_final.uns.copy()

In [40]:
import os

adata_train_final.write(os.path.join(output_dir, "adata_train_orig.h5ad"))
adata_ood_final.write(os.path.join(output_dir, "adata_ood_orig.h5ad"))
adata_test_final.write(os.path.join(output_dir, "adata_test_orig.h5ad"))

Check that reconstruction is meaningful

In [41]:
from sklearn.metrics import r2_score


def compute_r_squared(x: np.ndarray, y: np.ndarray) -> float:
    return r2_score(np.mean(x, axis=0), np.mean(y, axis=0))

In [42]:
decoded_test = np.matmul(adata_test_final.obsm["X_pca"], adata_train_final.varm["PCs"].T)

In [43]:
compute_r_squared(np.asarray(decoded_test + adata_test_final.varm["X_train_mean"].T), adata_test_final.X.A)

0.9993031270750916

In [44]:
decoded_ood = np.matmul(adata_ood_final.obsm["X_pca"], adata_train_final.varm["PCs"].T)

In [45]:
compute_r_squared(np.asarray(decoded_ood + adata_ood_final.varm["X_train_mean"].T), adata_ood_final.X.A)

0.9901804525345049

In [46]:
adata_train_final.uns.keys()

dict_keys(['cell_type_colors', 'hvg', 'neighbors', 'pathway1_colors', 'pca', 'rank_genes_groups_cov', 'rank_genes_groups_cov_detailed', 'source_colors', 'umap', 'log1p', 'rank_genes_groups_cov_all'])

In [47]:
adata_test_final.uns.keys()

dict_keys(['cell_type_colors', 'hvg', 'neighbors', 'pathway1_colors', 'pca', 'rank_genes_groups_cov', 'rank_genes_groups_cov_detailed', 'source_colors', 'umap', 'log1p', 'rank_genes_groups_cov_all'])

In [48]:
adata_train_final.obsm["X_pca"].shape

(49683, 30)

In [49]:
adata_test_final.obsm["X_pca"].shape

(5486, 30)

In [50]:
adata_ood_final.obsm["X_pca"].shape

(8351, 30)